In [ ]:
import pandas as pd
import time
import random
from collections import defaultdict, deque
from tqdm import tqdm

# =============================================================================
# Logger simples – use o flag debug_enabled para ativar/desativar mensagens
# =============================================================================
class DebugLogger:
    def __init__(self, max_lines=5, enable=True):
        self.max_lines = max_lines
        self.enable = enable

    def log(self, message):
        if self.enable:
            timestamp = time.strftime("%H:%M:%S")
            print(f"[{timestamp}] {message}")

# =============================================================================
# Funções para carregamento e pré-processamento dos dados
# =============================================================================
def load_data(caixas_path, estoque_path):
    """
    Carrega os dados das caixas e do estoque.
    """
    caixas_df = pd.read_csv(caixas_path)
    estoque_df = pd.read_csv(estoque_path)
    return caixas_df, estoque_df

def preprocess_data(caixas_df, estoque_df):
    """
    Converte os dataframes em listas/dicionários úteis para o algoritmo.
    Cada caixa passa a ter:
      - wave_class: a classe da onda (não se misturam)
      - caixa_id: identificador da caixa
      - pieces: número de peças
      - sku: identificação do produto (usado para alocar corredores)
    O estoque é organizado em um dicionário com listas ordenadas (decrescente pela quantidade).
    """
    boxes = []
    for _, row in caixas_df.iterrows():
        box = {
            'wave_class': row['CLASSE_ONDA'],
            'caixa_id': row['CAIXA_ID'],
            'pieces': row['PECAS'],
            'sku': row['SKU']
        }
        boxes.append(box)
    
    stock = defaultdict(list)
    for _, row in estoque_df.iterrows():
        stock[row['SKU']].append((row['ANDAR'], row['CORREDOR'], row['PECAS']))
    
    # Ordena o estoque para cada SKU pela quantidade (maior para menor)
    for sku in stock:
        stock[sku].sort(key=lambda x: -x[2])
    
    return boxes, stock

def allocate_sku(sku, required, stock):
    """
    Aloca os corredores para uma determinada SKU, conforme a necessidade (required).
    Se o estoque for insuficiente, gera um erro.
    Para cada peça requerida, inclui o par (andar, corredor).
    """
    allocated = []
    remaining = required
    for andar, corredor, qty in stock.get(sku, []):
        if remaining <= 0:
            break
        taken = min(qty, remaining)
        # Se quiser considerar cada peça individualmente, pode replicar o par
        allocated.extend([(andar, corredor)] * taken)
        remaining -= taken
    if remaining > 0:
        raise ValueError(f"Estoque insuficiente para SKU {sku}")
    return allocated

# =============================================================================
# Cálculo da área – função auxiliar
# =============================================================================
def calculate_contiguous(corredores, side):
    """
    Dado um conjunto de números de corredores (lado par ou ímpar),
    calcula o tamanho do maior bloco contíguo (considerando espaçamento de 2).
    """
    filtered = sorted([c for c in corredores if c % 2 == (0 if side == 'par' else 1)])
    if not filtered:
        return 0
    max_streak = current_streak = 1
    for i in range(1, len(filtered)):
        if filtered[i] == filtered[i-1] + 2:
            current_streak += 1
            max_streak = max(max_streak, current_streak)
        else:
            current_streak = 1
    return max_streak

# =============================================================================
# Classe que representa uma Onda (Wave)
# =============================================================================
class Wave:
    def __init__(self, wave_class, debug_enabled=False):
        self.debug_logger = DebugLogger(enable=debug_enabled)
        self.wave_class = wave_class
        self.boxes = []
        self.total_pieces = 0
        # Contagem de ocorrências de cada (andar, corredor)
        self.corridor_counts = defaultdict(int)

    def add_box(self, box):
        self.debug_logger.log(f"Adicionando caixa {box['caixa_id']} à onda {self.wave_class}")
        self.boxes.append(box)
        self.total_pieces += box['pieces']
        for (andar, corredor) in box['corridors']:
            self.corridor_counts[(andar, corredor)] += 1

    def remove_box(self, box):
        self.debug_logger.log(f"Removendo caixa {box['caixa_id']} da onda {self.wave_class}")
        if box in self.boxes:
            self.boxes.remove(box)
            self.total_pieces -= box['pieces']
            for (andar, corredor) in box['corridors']:
                self.corridor_counts[(andar, corredor)] -= 1
                if self.corridor_counts[(andar, corredor)] <= 0:
                    del self.corridor_counts[(andar, corredor)]
        else:
            self.debug_logger.log(f"Caixa {box['caixa_id']} não encontrada na onda {self.wave_class}")

    @property
    def corridors(self):
        return set(self.corridor_counts.keys())

    def area(self):
        """
        Calcula a "área" da onda da seguinte forma:
          - Para cada andar, agrupa os corredores em pares e ímpares (usando conjuntos para garantir
            que cada corredor seja contado apenas uma vez).
          - Para cada andar, a área é a soma do número de corredores pares distintos e do número de
            corredores ímpares distintos.
          - Se a onda utiliza mais de um andar, adiciona-se uma penalidade de 10 pontos para cada andar extra.
        
        Exemplo: Se em um mesmo andar temos 8 corredores pares distintos e 5 corredores ímpares distintos,
                 a área desse andar será 8 + 5 = 13.
        """
        # Agrupa os corredores por andar, separando entre pares e ímpares.
        floors = defaultdict(lambda: {'par': set(), 'impar': set()})
        for (floor, corridor) in self.corridor_counts:
            if corridor % 2 == 0:
                floors[floor]['par'].add(corridor)
            else:
                floors[floor]['impar'].add(corridor)
        
        total_area = 0
        for floor, sides in floors.items():
            # Para cada andar, conta quantos corredores distintos temos em cada lado.
            area_floor = len(sides['par']) + len(sides['impar'])
            total_area += area_floor
        
        # Penalidade: 10 pontos para cada andar extra (além do primeiro)
        penalty = 10 * (len(floors) - 1)
        return total_area + penalty


# =============================================================================
# Função GRASP para order batching (agrupamento em ondas)
# =============================================================================
def grasp_order_batching(boxes, stock, iterations=10, alpha=0.3, max_local_iterations=10, debug_enabled=False):
    global_logger = DebugLogger(enable=debug_enabled)
    global_logger.log("Iniciando processo GRASP")
    best_solution = None
    best_avg_area = float('inf')
    
    # Iterações da fase construtiva
    for it in tqdm(range(iterations), desc="Processando GRASP", unit="iter"):
        waves = []
        # Ordena as caixas pela quantidade de corredores (quanto mais corredores, maior a “dificuldade”)
        sorted_boxes = sorted(boxes, key=lambda x: -len(x['corridors']))
        
        for box in sorted_boxes:
            # Seleciona ondas viáveis: mesma classe e que não excedam 6000 peças
            feasible_waves = [w for w in waves if w.wave_class == box['wave_class'] and w.total_pieces + box['pieces'] <= 6000]
            candidates = []
            # Para cada onda viável, calcula a variação da área se a caixa for adicionada
            for wave in feasible_waves:
                original_area = wave.area()
                wave.add_box(box)
                new_area = wave.area()
                wave.remove_box(box)
                delta_area = new_area - original_area
                candidates.append((delta_area, wave))
            
            # Também considera criar uma nova onda contendo apenas a caixa
            new_wave = Wave(box['wave_class'], debug_enabled=debug_enabled)
            new_wave.add_box(box)
            candidates.append((new_wave.area(), new_wave))
            
            # Seleção aleatória dentro da lista restrita de candidatos (RCL)
            if candidates:
                min_val = min(delta for delta, _ in candidates)
                max_val = max(delta for delta, _ in candidates)
                threshold = min_val + alpha * (max_val - min_val)
                rcl = [candidate for candidate in candidates if candidate[0] <= threshold]
                delta, selected_wave = random.choice(rcl)
                
                # Se a onda escolhida já existir, adiciona a caixa; caso contrário, insere-a na lista de ondas
                if selected_wave in waves:
                    selected_wave.add_box(box)
                else:
                    waves.append(selected_wave)
            else:
                # Caso não haja candidatos (não deve ocorrer), cria nova onda
                new_wave = Wave(box['wave_class'], debug_enabled=debug_enabled)
                new_wave.add_box(box)
                waves.append(new_wave)
        
        # =============================================================================
        # Busca local: tenta melhorar a solução movendo caixas entre as ondas
        # =============================================================================
        improved = True
        local_iter = 0
        while improved and local_iter < max_local_iterations:
            local_iter += 1
            improved = False
            # Percorre cópia das ondas e caixas para evitar problemas com remoção durante iteração
            for wave in waves[:]:
                for box in wave.boxes[:]:
                    original_wave_area = wave.area()
                    wave.remove_box(box)
                    
                    best_delta = 0
                    best_target = None
                    
                    # Testa mover a caixa para cada onda viável (mesma classe, capacidade)
                    for target in waves:
                        if target != wave and target.wave_class == box['wave_class'] and target.total_pieces + box['pieces'] <= 6000:
                            original_target_area = target.area()
                            target.add_box(box)
                            new_target_area = target.area()
                            # Delta considerando a redução da área na onda original e no destino
                            delta_move = (new_target_area + wave.area()) - (original_target_area + original_wave_area)
                            target.remove_box(box)
                            
                            if delta_move < best_delta:
                                best_delta = delta_move
                                best_target = target
                    
                    # Testa a criação de uma nova onda com a caixa
                    new_target = Wave(box['wave_class'], debug_enabled=debug_enabled)
                    new_target.add_box(box)
                    delta_new = (new_target.area() + wave.area()) - original_wave_area
                    if delta_new < best_delta:
                        best_delta = delta_new
                        best_target = new_target
                    
                    # Se houver melhoria, realiza a mudança; caso contrário, retorna a caixa para sua onda original
                    if best_delta < 0 and best_target is not None:
                        if best_target not in waves:
                            waves.append(best_target)
                        best_target.add_box(box)
                        improved = True
                    else:
                        wave.add_box(box)
        
        # Calcula a média das áreas das ondas desta iteração
        total_area = sum(w.area() for w in waves)
        avg_area = total_area / len(waves) if waves else float('inf')
        if avg_area < best_avg_area:
            best_avg_area = avg_area
            best_solution = [w for w in waves]
        
        tqdm.write(f"Iteração {it+1}: Melhor área média = {best_avg_area:.2f} com {len(waves)} ondas.")
    
    return best_solution

# =============================================================================
# Função principal
# =============================================================================
def main():
    # Defina os caminhos dos arquivos (ajuste conforme sua estrutura de pastas)
    caixas_path = "../data/caixas.csv"
    estoque_path = "../data/estoque.csv"
    
    # Carrega os dados
    try:
        caixas_df, estoque_df = load_data(caixas_path, estoque_path)
    except Exception as e:
        print("Erro ao carregar os dados:", e)
        return
    
    # Pré-processa os dados
    boxes, stock = preprocess_data(caixas_df, estoque_df)
    
    # Para cada caixa, aloca os corredores disponíveis com base no estoque
    for box in boxes:
        try:
            box['corridors'] = allocate_sku(box['sku'], box['pieces'], stock)
        except ValueError as e:
            print(e)
            return  # Encerra se houver estoque insuficiente
    
    # Executa o GRASP – ajuste os parâmetros conforme necessário
    best_solution = grasp_order_batching(boxes, stock, iterations=10, alpha=0.3, debug_enabled=False)
    
    if best_solution is not None:
        total_avg = sum(w.area() for w in best_solution) / len(best_solution)
        print(f"\nMelhor área média encontrada: {total_avg:.2f}")
        for i, wave in enumerate(best_solution):
            print(f"\nOnda {i+1} (Classe: {wave.wave_class}):")
            print(f"Peças: {wave.total_pieces} | Área: {wave.area()}")
            print(f"Corredores: {wave.corridors}")
    else:
        print("Nenhuma solução foi encontrada.")

if __name__ == "__main__":
    main()